In [1]:
# Connect to Database
import os

database_url = os.environ["database_url"]
verbose = False

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

session = sessionmaker(bind=create_engine(database_url, echo=False), autoflush=False)()

In [2]:
# Select Daten aus dem Statistik Job

import json
from sqlalchemy import func
from kolonne.database.models import Job, JobHistorie

job, historie = session.query(Job, JobHistorie).join(JobHistorie).filter(Job.name == 'statistik').order_by(JobHistorie.zeitpunkt.desc()).first()
    
zeitpunkt = historie.zeitpunkt

data_historie = list(filter(lambda r: r and not r.startswith("#") and not r.startswith("*"), historie.ergebnis_text.split("\n")))[0]

data_historie = json.loads(data_historie)

if verbose:
    print(data_historie)

In [3]:
# Anzahl Aufmass im jeweiligen Status

from kolonne.database.models import get_status_str

data_status_count = data_historie["status_count"]
if verbose:
    print(data_status_count)
    
# Anzahl zusammenfassen

exclude_status = ["A"]

data = dict()
for d in filter(lambda d: d["status"] not in exclude_status, data_status_count):
    status = get_status_str(d["status"])
    current = data.get(status, 0)
    data[status] = current + d["status__count"]

if verbose:
    print(data)

In [4]:
columns = ["Status", "Anzahl"]

data_csv = list(data.items())

if verbose:
    print(data_csv)   

In [5]:
# DataFrame erstellen

import pandas as pd

data_frame = pd.DataFrame.from_records(data_csv, columns=columns)

if verbose:
    print(data_frame)


In [ ]:
# Diagramm erstellen

import matplotlib.pyplot as plt
import seaborn as sn

sn.set(font_scale=1.0)
fig, ax = plt.subplots(figsize=(20,7))

# labels = data_frame.loc[:,'Status'].values

labels = [f"{d[0]}: {d[1]}" for d in data_csv]
anzahl = data_frame.loc[:,'Anzahl'].values


ax.pie(anzahl, labels=labels,
       shadow=False, startangle=0,
       pctdistance=1.2,labeldistance=1.4)
ax.axis('equal')

ax.set_title(f"Aufmaß Status {zeitpunkt}", y=1.3, fontsize = 16)
ax.legend(frameon=True, bbox_to_anchor=(1,0.8))

# Kontextmenü "Create New View for Output" um das Bild zu speichern